In [1]:
import dash
dash.__version__

'2.0.0'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as re
from tqdm import tqdm
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash 
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output,State
from dash import dash_table

1. Task:

In [3]:


# All the sorted 101 symbols of the S&P 100
Symbols= ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN", "AVGO", "AXP", "BA",
         "BAC", "BK", "BKNG", "BLK", "BMY", "BRK-B", "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP", "COST",
         "CRM", "CSCO", "CVS", "CVX", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD", 
         "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KO", "LIN", "LLY", "LMT",
         "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "META", "MMM", "MO", "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE", 
         "NVDA", "ORCL", "PEP", "PFE", "PG", "PM", "PYPL", "QCOM", "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", 
         "TMO", "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WBA", "WFC", "WMT", "XOM"
        ]

 
base_url='https://financialmodelingprep.com/api/v3/profile/'  
api_key = '93c0b92d77a3cbb8927f893eaf18e1c5'
params = {'apikey': api_key}

SP100_companies = pd.DataFrame()
for symbol in tqdm(Symbols):
    response = re.get(base_url+symbol, params=params)
    if response.status_code != 200 :
        print('Error!')
        break
    df = pd.DataFrame.from_dict(response.json(), orient='columns')
    SP100_companies = pd.concat([SP100_companies, df], ignore_index=True)
    del df
    

SP100_companies.head(5)


100%|██████████| 101/101 [01:41<00:00,  1.00s/it]


,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,AAPL,164.975,1.297088,65275362,2610217915546,0.920000,124.17-176.15,-0.585,Apple Inc.,USD,...,95014,4.151760,150.082,https://financialmodelingprep.com/image-stock/...,1980-12-12,False,False,True,False,False
1,ABBV,161.450,0.586811,6135344,284820408166,5.920000,134.09-168.11,-1.240,AbbVie Inc.,USD,...,60064-6400,16.351400,162.631,https://financialmodelingprep.com/image-stock/...,2013-01-02,False,False,True,False,False
2,ABT,103.740,0.665731,5242919,180294928020,2.040000,93.25-124.35,0.130,Abbott Laboratories,USD,...,60064-6400,0.583438,109.367,https://financialmodelingprep.com/image-stock/...,1937-03-01,False,False,True,False,False
3,ACN,278.675,1.241669,2461624,175992189018,4.480000,242.8-330.32,-6.595,Accenture plc,USD,...,2,22.396000,299.666,https://financialmodelingprep.com/image-stock/...,2009-09-01,False,False,True,False,False
4,ADBE,378.840,1.271483,3053414,173773908000,0.356125,274.73-451.15,0.040,Adobe Inc.,USD,...,95110-2704,41.228300,329.482,https://financialmodelingprep.com/image-stock/...,1986-01-08,False,False,True,False,False


In [4]:
SP100_companies_subset= SP100_companies[['companyName','symbol','sector']].reset_index(drop=True)

# Set the seed to my matriculation number = "103591"
np.random.seed(103591)

# Generate 20 unique random integers between 0 and 101
random_numbers = np.random.choice(101, size=20, replace=False) 

# to subset the original DataFrame based on the generated random numbers and then to sort its indexes sort_index() is used.
random_SP100_companies= SP100_companies_subset.loc[random_numbers].sort_index()
random_SP100_companies

,companyName,symbol,sector
0,Apple Inc.,AAPL,Technology
6,"Advanced Micro Devices, Inc.",AMD,Technology
15,Booking Holdings Inc.,BKNG,Consumer Cyclical
17,Bristol-Myers Squibb Company,BMY,Healthcare
18,Berkshire Hathaway Inc.,BRK-B,Financial Services
24,Capital One Financial Corporation,COF,Financial Services
25,ConocoPhillips,COP,Energy
27,"Salesforce, Inc.",CRM,Technology
32,The Walt Disney Company,DIS,Communication Services
45,"The Goldman Sachs Group, Inc.",GS,Financial Services


 2.Task:
 

In [5]:
# Generating the ticker list of the 20 randomly chosen companies and converting them to a list
ticker_list= random_SP100_companies['symbol'].tolist()

# Loading the historical open, adjusted close, high, low and volume of the the unique companies dataset 
start = '2015-01-01'
end = '2022-12-31'
my_company_data = yf.download(ticker_list, start=start, end=end)
# The data fetched by the yfinance has mutlidimensional columns and to make the work with data frames easier 
# a dictionary is used to store the dataframes for each ticker symbol,
#where the keys of the dictionary are the ticker symbols and the values are the corresponding dataframes. 
data_dict = {}
for ticker in ticker_list:
    data_dict[f"data_{ticker}"]=pd.DataFrame({'Open':my_company_data[('Open',ticker)],
                           'High': my_company_data[('High',ticker)],
                           'Low': my_company_data[('Low',ticker)],
                           'Close': my_company_data[('Close',ticker)],
                           'Adj Close': my_company_data[('Adj Close',ticker)],
                           'Volume': my_company_data[('Volume',ticker)]}
                         )
    data_dict[f"data_{ticker}"]['daily_discrete_return'] = data_dict[f"data_{ticker}"] ['Adj Close'].pct_change()
    data_dict[f"data_{ticker}"]['daily_log_return']= np.log(data_dict[f"data_{ticker}"]['Adj Close']).diff()

[*********************100%***********************]  20 of 20 completed


In [6]:
# Loading the historical open, adjusted close, high, low and volume of the S&P 100 index
start = '2015-01-01'
end = '2022-12-31'
SP100_index_data= yf.download("^OEX", start=start, end=end)

# Calculate the daily discrete and log return of S&P 100
SP100_index_data['daily_discrete_return'] = SP100_index_data ['Adj Close'].pct_change()
SP100_index_data['daily_log_return'] = np.log(SP100_index_data ['Adj Close']).diff()

SP100_index_data


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,daily_discrete_return,daily_log_return
Date,,,,,,,,
2015-01-02,908.380005,914.460022,902.700012,907.500000,907.500000,2708700000,NaN,NaN
2015-01-05,905.570007,905.570007,889.820007,891.039978,891.039978,3799120000,-0.018138,-0.018304
2015-01-06,891.890015,895.669983,879.340027,883.960022,883.960022,4460110000,-0.007946,-0.007977
2015-01-07,885.099976,895.619995,885.099976,893.599976,893.599976,3805480000,0.010905,0.010846
2015-01-08,895.739990,911.070007,895.739990,909.869995,909.869995,3934010000,0.018207,0.018044
...,...,...,...,...,...,...,...,...
2022-12-23,1696.910034,1711.560059,1688.339966,1710.709961,1710.709961,2819280000,0.005584,0.005569
2022-12-27,1709.010010,1710.010010,1693.560059,1700.140015,1700.140015,3030300000,-0.006179,-0.006198
2022-12-28,1699.770020,1709.979980,1679.239990,1680.540039,1680.540039,3083520000,-0.011528,-0.011595


3.Task:


In [7]:
#creating a pandas DataFrame containing the Adj Close price of all companies and the S&P 100 index to calculate the correlation

companies_close_price=pd.DataFrame({'SP100': SP100_index_data['Adj Close']})
for ticker in ticker_list:
    df= pd.DataFrame({ticker :data_dict[f"data_{ticker}"]['Adj Close']})
    companies_close_price=pd.concat([companies_close_price,df], axis=1)
    del df
companies_close_price.head()

,SP100,AAPL,AMD,BKNG,BMY,BRK-B,COF,COP,CRM,DIS,...,LLY,MDT,MS,PM,T,TGT,TMO,UPS,V,VZ
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,907.500000,24.565693,2.67,1142.060059,46.971115,149.169998,70.821655,52.687008,59.240002,87.376251,...,58.668423,59.794415,31.525970,53.161453,13.416445,60.022522,124.950455,85.914284,62.620525,31.495939
2015-01-05,891.039978,23.873648,2.66,1097.579956,46.237076,147.000000,69.190414,50.179565,58.169998,86.099396,...,58.090633,59.544857,30.540537,52.761189,13.289684,58.946838,123.234001,84.194122,61.238266,31.234371
2015-01-06,883.960022,23.875889,2.63,1079.959961,46.158134,146.839996,67.902573,48.107868,57.200001,85.642715,...,58.383732,59.245403,29.660957,53.213943,13.309494,58.938869,122.083183,83.641502,60.843662,31.549597
2015-01-07,893.599976,24.210682,2.58,1069.569946,46.758011,148.880005,68.675262,48.428944,56.930000,86.518806,...,57.973400,60.726124,30.084461,53.614201,13.387097,61.169891,125.740417,84.419884,61.658848,31.346001
2015-01-08,909.869995,25.140905,2.61,1082.849976,47.926167,151.369995,69.344925,49.636787,58.590000,87.413544,...,59.346729,62.181866,30.532393,54.421265,13.520282,61.456749,127.047264,85.937645,62.485851,32.017849


In [8]:
# The Bravais Pearson is used to calculate correlation between each the Adj close price of each stock and the S&P 100 index 
#the results are saved in a correlation matrix and then the values are sorted from the lowest to the highest.
corr_matrix= companies_close_price.corr(method= "pearson")['SP100'].sort_values()
corr_matrix

T        0.357718
COP      0.583701
BMY      0.645065
PM       0.708231
VZ       0.724082
DIS      0.738077
BKNG     0.810272
COF      0.862773
MDT      0.870868
LLY      0.875581
GS       0.902215
CRM      0.908360
V        0.937932
BRK-B    0.948032
TGT      0.950314
UPS      0.951287
MS       0.953684
AMD      0.967430
AAPL     0.970554
TMO      0.979418
SP100    1.000000
Name: SP100, dtype: float64

In [9]:
# Has the highest correlation values
high_corr= corr_matrix[10:20].index.tolist()
high_corr


['GS', 'CRM', 'V', 'BRK-B', 'TGT', 'UPS', 'MS', 'AMD', 'AAPL', 'TMO']

In [10]:
weights= np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]) # Equal weight of assets

In [11]:
portfolio_daily_high_corr= pd.DataFrame()
for ticker in high_corr:
    df= pd.DataFrame({f"daily_discrete_{ticker}": data_dict[f"data_{ticker}"]['daily_discrete_return']})
    portfolio_daily_high_corr= pd.concat([portfolio_daily_high_corr,df],axis=1)
    del df
    
    
# Calculating the daily portfolio return based on the weighted sum of the individual stock returns
portfolio_daily_high_corr['daily_discrete_PF_return'] = (portfolio_daily_high_corr * weights).sum(axis=1) 

# Calculating the daily log return of the portfolio
portfolio_daily_high_corr['daily_log_PF_return'] = np.log(1 + portfolio_daily_high_corr['daily_discrete_PF_return'])

portfolio_daily_high_corr.head()

,daily_discrete_GS,daily_discrete_CRM,daily_discrete_V,daily_discrete_BRK-B,daily_discrete_TGT,daily_discrete_UPS,daily_discrete_MS,daily_discrete_AMD,daily_discrete_AAPL,daily_discrete_TMO,daily_discrete_PF_return,daily_log_PF_return
Date,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2015-01-05,-0.031223,-0.018062,-0.022074,-0.014547,-0.017921,-0.020022,-0.031258,-0.003745,-0.028171,-0.013737,-0.020076,-0.020280
2015-01-06,-0.020229,-0.016675,-0.006444,-0.001088,-0.000135,-0.006564,-0.028800,-0.011278,0.000094,-0.009338,-0.010046,-0.010097
2015-01-07,0.014903,-0.004720,0.013398,0.013893,0.037853,0.009306,0.014278,-0.019011,0.014022,0.029957,0.012388,0.012312
2015-01-08,0.015965,0.029159,0.013413,0.016725,0.004690,0.017979,0.014889,0.011628,0.038422,0.010393,0.017326,0.017178


In [12]:
# To caclulte the monthly discrete and log returns of each portfolio a sub portfolio is created that has only the monthly returns
# Resampling the daily data to monthly data
df = portfolio_daily_high_corr[['daily_discrete_PF_return','daily_log_PF_return']]
portfolio_monthly_high_corr=pd.DataFrame()
# Calculating the monthly log return of the portfolio first the data is grouped into a month frequency and 
# then the daily log returns are summed up 
portfolio_monthly_high_corr['monthly_log_PF_return'] = df['daily_log_PF_return'].resample('M').sum()
# Calculating the monthly discrete return of the portfolio 
portfolio_monthly_high_corr['monthly_discrete_PF_return'] = np.exp(portfolio_monthly_high_corr['monthly_log_PF_return']) - 1

portfolio_monthly_high_corr.head()

,monthly_log_PF_return,monthly_discrete_PF_return
Date,,
2015-01-31,-0.047476,-0.046367
2015-02-28,0.088625,0.092671
2015-03-31,-0.022563,-0.022310
2015-04-30,-0.004586,-0.004576
2015-05-31,0.022855,0.023118


In [13]:
# Portfolio with the lowest correlation values
low_corr=corr_matrix[0:10].index.tolist()
low_corr



['T', 'COP', 'BMY', 'PM', 'VZ', 'DIS', 'BKNG', 'COF', 'MDT', 'LLY']

In [14]:

# Calculating the daily discrete return and the daily log return for low portfolio:
portfolio_daily_low_corr= pd.DataFrame()  
for ticker in low_corr:
    df= pd.DataFrame({f"daily_discrete_{ticker}": data_dict[f"data_{ticker}"]['daily_discrete_return']})
    portfolio_daily_low_corr= pd.concat([portfolio_daily_low_corr,df],axis=1)
    del df
    
# Calculate the daily portfolio return based on the weighted sum of the individual stock returns
portfolio_daily_low_corr['daily_discrete_PF_return'] = (portfolio_daily_low_corr * weights).sum(axis=1)  
# Calculating the daily log return of the portfolio
portfolio_daily_low_corr['daily_log_PF_return'] = np.log(1 + portfolio_daily_low_corr['daily_discrete_PF_return'])

portfolio_daily_low_corr.head(5)

,daily_discrete_T,daily_discrete_COP,daily_discrete_BMY,daily_discrete_PM,daily_discrete_VZ,daily_discrete_DIS,daily_discrete_BKNG,daily_discrete_COF,daily_discrete_MDT,daily_discrete_LLY,daily_discrete_PF_return,daily_log_PF_return
Date,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2015-01-05,-0.009448,-0.047591,-0.015627,-0.007529,-0.008305,-0.014613,-0.038947,-0.023033,-0.004174,-0.009848,-0.017912,-0.018074
2015-01-06,0.001491,-0.041286,-0.001707,0.008581,0.010092,-0.005304,-0.016053,-0.018613,-0.005029,0.005046,-0.006278,-0.006298
2015-01-07,0.005831,0.006674,0.012996,0.007522,-0.006453,0.010230,-0.009621,0.011379,0.024993,-0.007028,0.005652,0.005636
2015-01-08,0.009949,0.024941,0.024983,0.015053,0.021433,0.010342,0.012416,0.009751,0.023972,0.023689,0.017653,0.017499


In [15]:
# To caclulte the monthly discrete and log returns of each portfolio a sub portfolio is created that has only the monthly returns
df = portfolio_daily_low_corr[['daily_discrete_PF_return','daily_log_PF_return']]
portfolio_monthly_low_corr=pd.DataFrame()
# Calculate the monthly log return of the portfolio 
portfolio_monthly_low_corr['monthly_log_PF_return'] = df['daily_log_PF_return'].resample('M').sum()
# Calculate the monthly discrete return of the portfolio
portfolio_monthly_low_corr['monthly_discrete_PF_return'] = np.exp(portfolio_monthly_low_corr['monthly_log_PF_return']) - 1

portfolio_monthly_low_corr.head()

,monthly_log_PF_return,monthly_discrete_PF_return
Date,,
2015-01-31,-0.035368,-0.034750
2015-02-28,0.071042,0.073626
2015-03-31,-0.014632,-0.014526
2015-04-30,0.038572,0.039325
2015-05-31,0.006398,0.006419


In [16]:
# Creating a DataFrame that has the daily discrete return of each stock to calculate its volatility
companies_vol = pd.DataFrame()
for ticker in ticker_list:
    df= pd.DataFrame({f"daily_discrete_{ticker}": data_dict[f"data_{ticker}"]['daily_discrete_return']})
    companies_vol = pd.concat([companies_vol,df],axis=1)
    del df
    
companies_vol.head()

,daily_discrete_AAPL,daily_discrete_AMD,daily_discrete_BKNG,daily_discrete_BMY,daily_discrete_BRK-B,daily_discrete_COF,daily_discrete_COP,daily_discrete_CRM,daily_discrete_DIS,daily_discrete_GS,daily_discrete_LLY,daily_discrete_MDT,daily_discrete_MS,daily_discrete_PM,daily_discrete_T,daily_discrete_TGT,daily_discrete_TMO,daily_discrete_UPS,daily_discrete_V,daily_discrete_VZ
Date,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.028171,-0.003745,-0.038947,-0.015627,-0.014547,-0.023033,-0.047591,-0.018062,-0.014613,-0.031223,-0.009848,-0.004174,-0.031258,-0.007529,-0.009448,-0.017921,-0.013737,-0.020022,-0.022074,-0.008305
2015-01-06,0.000094,-0.011278,-0.016053,-0.001707,-0.001088,-0.018613,-0.041286,-0.016675,-0.005304,-0.020229,0.005046,-0.005029,-0.028800,0.008581,0.001491,-0.000135,-0.009338,-0.006564,-0.006444,0.010092
2015-01-07,0.014022,-0.019011,-0.009621,0.012996,0.013893,0.011379,0.006674,-0.004720,0.010230,0.014903,-0.007028,0.024993,0.014278,0.007522,0.005831,0.037853,0.029957,0.009306,0.013398,-0.006453
2015-01-08,0.038422,0.011628,0.012416,0.024983,0.016725,0.009751,0.024941,0.029159,0.010342,0.015965,0.023689,0.023972,0.014889,0.015053,0.009949,0.004690,0.010393,0.017979,0.013413,0.021433


In [17]:
# Calculating the realised volatility for each company
realised_vol = companies_vol.std().sort_values()
realised_vol

daily_discrete_VZ       0.011734
daily_discrete_BRK-B    0.012869
daily_discrete_T        0.014110
daily_discrete_MDT      0.014925
daily_discrete_PM       0.015164
daily_discrete_BMY      0.015573
daily_discrete_TMO      0.016048
daily_discrete_UPS      0.016265
daily_discrete_V        0.016428
daily_discrete_LLY      0.017285
daily_discrete_DIS      0.017797
daily_discrete_GS       0.018728
daily_discrete_AAPL     0.018866
daily_discrete_TGT      0.019744
daily_discrete_MS       0.020515
daily_discrete_BKNG     0.021071
daily_discrete_CRM      0.022304
daily_discrete_COF      0.023423
daily_discrete_COP      0.025780
daily_discrete_AMD      0.038509
dtype: float64

In [18]:
# tickers with a high volatility 
high_vol =['DIS', 'GS', 'AAPL', 'TGT', 'MS', 'BKNG', 'CRM', 'COF', 'COP', 'AMD']
portfolio_daily_high_vol= pd.DataFrame()
for ticker in high_vol:
    df= companies_vol[f"daily_discrete_{ticker}"]
    portfolio_daily_high_vol =pd.concat([portfolio_daily_high_vol,df],axis=1)
    del df
portfolio_daily_high_vol.index= pd.to_datetime(portfolio_daily_high_vol.index)

# Calculate the daily portfolio return based on the weighted sum of the individual stock returns
portfolio_daily_high_vol['daily_discrete_PF_return'] = (portfolio_daily_high_vol * weights).sum(axis=1)  
# Calculating the daily log return of the portfolio
portfolio_daily_high_vol['daily_log_PF_return'] = np.log(1 + portfolio_daily_high_vol['daily_discrete_PF_return'])


portfolio_daily_high_vol.head()


,daily_discrete_DIS,daily_discrete_GS,daily_discrete_AAPL,daily_discrete_TGT,daily_discrete_MS,daily_discrete_BKNG,daily_discrete_CRM,daily_discrete_COF,daily_discrete_COP,daily_discrete_AMD,daily_discrete_PF_return,daily_log_PF_return
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2015-01-05,-0.014613,-0.031223,-0.028171,-0.017921,-0.031258,-0.038947,-0.018062,-0.023033,-0.047591,-0.003745,-0.025457,-0.025786
2015-01-06,-0.005304,-0.020229,0.000094,-0.000135,-0.028800,-0.016053,-0.016675,-0.018613,-0.041286,-0.011278,-0.015828,-0.015955
2015-01-07,0.010230,0.014903,0.014022,0.037853,0.014278,-0.009621,-0.004720,0.011379,0.006674,-0.019011,0.007599,0.007570
2015-01-08,0.010342,0.015965,0.038422,0.004690,0.014889,0.012416,0.029159,0.009751,0.024941,0.011628,0.017220,0.017074


In [19]:
# To caclulte the monthly discrete and log returns of each portfolio a sub portfolio is created that has only the monthly returns
df = portfolio_daily_high_vol[['daily_discrete_PF_return','daily_log_PF_return']]
portfolio_monthly_high_vol=pd.DataFrame()
# Calculating the monthly log return of the portfolio first the data is grouped into a month frequency and 
# then the daily log returns are summed up 
portfolio_monthly_high_vol['monthly_log_PF_return'] = df['daily_log_PF_return'].resample('M').sum()
# Calculate the monthly discrete return of the portfolio
portfolio_monthly_high_vol['monthly_discrete_PF_return'] = np.exp(portfolio_monthly_high_vol['monthly_log_PF_return']) - 1

portfolio_monthly_high_vol.head()

,monthly_log_PF_return,monthly_discrete_PF_return
2015-01-31,-0.063139,-0.061187
2015-02-28,0.119278,0.126683
2015-03-31,-0.025052,-0.024741
2015-04-30,0.020547,0.020760
2015-05-31,0.009551,0.009597


In [20]:
# tickers with a low volatility 
low_vol = ['VZ', 'BRK-B', 'T', 'MDT', 'PM', 'BMY', 'TMO', 'UPS', 'V', 'LLY']

portfolio_daily_low_vol= pd.DataFrame()  # this portfolio has the discrete daily stocks returns
for ticker in low_vol:
    df= companies_vol[f"daily_discrete_{ticker}"]
    portfolio_daily_low_vol =pd.concat([portfolio_daily_low_vol,df],axis=1)
    del df
portfolio_daily_low_vol.index= pd.to_datetime(portfolio_daily_low_vol.index)
    
# Calculate the daily portfolio return based on the weighted sum of the individual stock returns
portfolio_daily_low_vol['daily_discrete_PF_return'] = (portfolio_daily_low_vol * weights).sum(axis=1)  
# Calculating the daily log return of the portfolio
portfolio_daily_low_vol['daily_log_PF_return'] = np.log(1 + portfolio_daily_low_vol['daily_discrete_PF_return'])

portfolio_daily_low_vol.head()

,daily_discrete_VZ,daily_discrete_BRK-B,daily_discrete_T,daily_discrete_MDT,daily_discrete_PM,daily_discrete_BMY,daily_discrete_TMO,daily_discrete_UPS,daily_discrete_V,daily_discrete_LLY,daily_discrete_PF_return,daily_log_PF_return
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2015-01-05,-0.008305,-0.014547,-0.009448,-0.004174,-0.007529,-0.015627,-0.013737,-0.020022,-0.022074,-0.009848,-0.012531,-0.012610
2015-01-06,0.010092,-0.001088,0.001491,-0.005029,0.008581,-0.001707,-0.009338,-0.006564,-0.006444,0.005046,-0.000496,-0.000496
2015-01-07,-0.006453,0.013893,0.005831,0.024993,0.007522,0.012996,0.029957,0.009306,0.013398,-0.007028,0.010441,0.010387
2015-01-08,0.021433,0.016725,0.009949,0.023972,0.015053,0.024983,0.010393,0.017979,0.013413,0.023689,0.017759,0.017603


In [21]:
# Resample the daily data to monthly data
df = portfolio_daily_low_vol[['daily_discrete_PF_return','daily_log_PF_return']]
portfolio_monthly_low_vol=pd.DataFrame()
# Calculate the monthly log return of the portfolio
portfolio_monthly_low_vol['monthly_log_PF_return'] = df['daily_log_PF_return'].resample('M').sum()
# Calculate the monthly discrete return of the portfolio
portfolio_monthly_low_vol['monthly_discrete_PF_return'] = np.exp(portfolio_monthly_low_vol['monthly_log_PF_return']) - 1

portfolio_monthly_low_vol.head()

,monthly_log_PF_return,monthly_discrete_PF_return
2015-01-31,-0.019844,-0.019648
2015-02-28,0.040312,0.041136
2015-03-31,-0.012158,-0.012085
2015-04-30,0.013312,0.013401
2015-05-31,0.019649,0.019844


4.Task

In [22]:
#Calculating the Sharpe Ratio for SP100 index
SP_mean = SP100_index_data['daily_discrete_return'].resample("Y").mean()
SP_std = SP100_index_data['daily_discrete_return'].resample("Y").std()
SR_SP100= np.round(SP_mean/SP_std,4)

# Defining the function to calculate Sharpe Ratio of the portfolios
def calculate_SR(df):
    df_mean = df['daily_discrete_PF_return'].resample("Y").mean()
    df_std = df['daily_discrete_PF_return'].resample("Y").std()
    SR = np.round(df_mean / df_std,4)
    return SR
# Calculating the Sharpe Ratios for each portfolio
SR_high_corr = calculate_SR(portfolio_daily_high_corr)
SR_low_corr = calculate_SR(portfolio_daily_low_corr)
SR_high_vol = calculate_SR(portfolio_daily_high_vol)
SR_low_vol = calculate_SR(portfolio_daily_low_vol)
# Combine the Sharpe Ratios into one dataframe
df_SR = pd.concat([SR_high_corr, SR_low_corr, SR_high_vol, SR_low_vol, SR_SP100], axis=1)
df_SR.columns = ['SR_high_corr', 'SR_low_corr', 'SR_high_vol', 'SR_low_vol', 'SR_SP100']





# The App's tabs
app_tabs= html.Div([
    dbc.Tabs(
        [
        dbc.Tab(label="Summary", tab_id="tab_summary", ),
        dbc.Tab(label="Correlation", tab_id="tab_corr",),
        dbc.Tab(label="Volatility", tab_id="tab_vol",)
        ],id="tabs", active_tab="tab_summary")
    ],className="mt-1")
# to make the app looks more structured Bootstrap is used  
app = JupyterDash (__name__,external_stylesheets=[dbc.themes.QUARTZ], suppress_callback_exceptions=True)
app.layout = dbc.Container([
    
    dbc.Row(dbc.Col(html.H1("Financial Data Analytics Dashboard",style={"textAlign":"center", "marginTop":"30px"}),width=12)),
    html.Hr(),
    dbc.Row(dbc.Col(app_tabs,width=12),className="mb-3"),
    html.Div(id="content", children=[])
             ])
###
# This function displays the content of the summary tab
def get_summary_content():
    portfolios=['portfolio_daily_high_corr', 'portfolio_daily_low_corr',
                'portfolio_daily_high_vol','portfolio_daily_low_vol']

    layout= dbc.Container([
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        dcc.Dropdown(
                             id="dropdown_3",
                             options=[{'label': i ,'value':i } for i in portfolios ],
                             placeholder= "Please select...",
                             clearable= True,
                             value='portfolio_daily_high_corr',
                             className="bg-white bg-opacity-25 text-dark border-0 rounded-lg shadow-sm form-select"
                             
                        ),
                        dcc.Graph(id="graph_1"),
                         # Add checkbox to toggle SP100 index data
                        dcc.Checklist(
                            id="checkbox_sp100",
                            options=[{"label": "Show SP100 Index data", "value": "show_sp100"}],
                            value=[],
                        ),
                        
                      ])
                )
            ],width=6),
            
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                         dcc.Dropdown(
                             id="dropdown_4",
                             options=[{'label': i ,'value':i } for i in portfolios ],
                             placeholder= "Please select...",
                             clearable= True,
                             value='portfolio_daily_high_corr',
                             className="bg-white bg-opacity-25 text-dark border-0 rounded-lg shadow-sm form-select"
                             
                         ),
                        
                         dcc.Graph(id="graph_2")
                    
                    ])
                )
            ],width=6)
        ]),
        dbc.Row([
            dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Sharpe Ratios"),
                            dbc.CardBody(
                                dash_table.DataTable(
                                    id='table',
                                    columns=[{"name": i, "id": i} for i in df_SR.columns],
                                    data=df_SR.reset_index().to_dict('records'),
                                    style_table={'overflowX': 'auto'},
                                    style_cell={'color': 'black'}
                                    
                                )
                            )
                        ],
                        style={'marginTop': 20}
                    )
            )
        ])
    ])
    return layout



###
# Getting the names of the companies corresponding to the ticker symbols in high_corr
company_names_high_corr = [SP100_companies.loc[SP100_companies['symbol'] == ticker, 'companyName'].values[0] 
                 for ticker in high_corr]
 # Getting the names of the companies corresponding to the ticker symbols in _low_corr   
company_names_low_corr = [SP100_companies.loc[SP100_companies['symbol'] == ticker, 'companyName'].values[0] 
                 for ticker in low_corr]
#This function displays the content of the correlation tab
def get_correlation_content():
    
    bullet_list_1 = [html.Li(name) for name in company_names_high_corr]
    bullet_list_2 = [html.Li(name) for name in company_names_low_corr]
    
    company_names = company_names_high_corr + company_names_low_corr
    
    layout= dbc.Container([
        dbc.Row(
            [
            dbc.Col (
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H5("High correlated companies:"),
                            html.Ul(bullet_list_1)  # Wrap the bullet list in an <ul> tag
                        ]
                    ),
                className="mb-3"),
            width=6),
                
            dbc.Col (
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H5("Low correlated companies:"),
                            html.Ul(bullet_list_2)  
                        ]
                        
                    ),
                className="mb-3"),
            width=6)
                    
            ]
        ),
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                id="dropdown_1",
                options=[{'label': i ,'value':i } for i in company_names ],
                placeholder= "Please select...",
                clearable= True,
                optionHeight=25,
                className="bg-white bg-opacity-25 text-dark border-0 rounded-lg shadow-sm form-select"
                
                )
                
            ],width= 12, className="mb-3")  
        ],className="justify-content-left"),
        
        dbc.Row([
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                        html.Img(id="company-image_1", className="mb-3",
                            style={
                                "height": "150px",         
                                 "width": "auto",              
                                 "display": "block",        
                                 "margin-left": "auto",     
                                 "margin-right": "auto",    
                                 "text-align": "center"
                                 }),
                        html.P(id="company_membership_1", className="card-text"),
                        html.P(id="company-sector_1", className="card-text"),
                        html.P(id="company-mktCap_1", className="card-text"),
                        html.P(id="company-employees_1", className="card-text"),
                        html.A("Visit Website", id="company-website_1", className="btn btn-primary",
                               target="_blank", style={"display": "none"})
                        ],style={"text-align": "center"})
                    ])
                ),width=12),
            
        ])   
    ])
    return layout




###    
# Getting the names of the companies corresponding to the ticker symbols in ticker_high_vol
company_names_high_vol = [SP100_companies.loc[SP100_companies['symbol'] == ticker, 'companyName'].values[0] 
                 for ticker in high_vol]
# Getting the names of the companies corresponding to the ticker symbols in ticker_low_vol
company_names_low_vol = [SP100_companies.loc[SP100_companies['symbol'] == ticker, 'companyName'].values[0] 
                 for ticker in low_vol]
#This function displays the content of the volatility tab
def get_volatility_content():
    bullet_list_1 = [html.Li(name) for name in company_names_high_vol]
    bullet_list_2 = [html.Li(name) for name in company_names_low_vol]
    
    company_names = company_names_high_vol + company_names_low_vol
    layout= dbc.Container([
        dbc.Row(
            [
            dbc.Col (
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H5("Companies with a high volatility:"),
                            html.Ul(bullet_list_1)  # Wrap the bullet list in an <ul> tag
                        ]
                    ),
                className="mb-3"),
            width=6),
                
            dbc.Col (
                dbc.Card(
                    dbc.CardBody(
                        [
                            html.H5("Companies with a low Volatility:"),
                            html.Ul(bullet_list_2)  
                        ]   
                    ),
                className="mb-3"),
            width=6)
                    
            ]
        ),
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                id="dropdown_2",
                options=[{'label': i ,'value':i } for i in company_names ],
                placeholder= "Please select...",
                clearable= True,
                optionHeight=25,
                className="bg-white bg-opacity-25 text-dark border-0 rounded-lg shadow-sm form-select"
                )],
                width= 12, className="mb-3")],  
         className="justify-content-left"),
        dbc.Row([
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                        html.Img(id="company-image_2", className="mb-3",
                            style={
                                "height": "100px",         
                                 "width": "auto",                  
                                 "display": "block",        
                                 "margin-left": "auto",     
                                 "margin-right": "auto",    
                                 "text-align": "center"
                                 }),
                        html.P(id="company_membership_2", className="card-text"),
                        html.P(id="company-sector_2", className="card-text"),
                        html.P(id="company-mktCap_2", className="card-text"),
                        html.P(id="company-employees_2", className="card-text"),
                        html.A("Visit Website", id="company-website_2", className="btn btn-primary ",
                               target="_blank", style={"display": "none"})
                        ],style={"text-align": "center"})
                    ])
                ),width=12),  
        ])  
    ])
    return layout




###
# To switch between tabs
@app.callback(
      Output("content", "children"),
      Input("tabs", "active_tab"))
def switch_tab(tab_chosen):
    if tab_chosen=="tab_summary":
        return get_summary_content()
    elif tab_chosen =="tab_corr":
        return get_correlation_content()
    return  get_volatility_content()


###
# To display companies informations related to the correlation portfolios
@app.callback(
    [Output("company-image_1", "src"),
     Output("company_membership_1", "children"),
     Output("company-sector_1", "children"),
     Output("company-mktCap_1", "children"),
     Output("company-employees_1", "children"),
     Output("company-website_1", "href")],
    [Input("dropdown_1", "value")])
def company_info_corr_tab (value):
    if value is None:
        return [None,None,None,None,None,None]
    else:
        if value in company_names_high_corr:
            membership= "Portfolio with high correlation"
        elif value in company_low_corr:
            membership= "Portfolio with low correlation"
        image = SP100_companies.image[(SP100_companies.companyName)== value].values[0]
        sector = SP100_companies.sector[(SP100_companies.companyName)== value].values[0]
        mktCap = SP100_companies.mktCap[(SP100_companies.companyName)== value].values[0]
        nb_ofemployees = SP100_companies.fullTimeEmployees[(SP100_companies.companyName)== value].values[0]
        website = SP100_companies.website[(SP100_companies.companyName)== value].values[0]
        return [image, f"Company membership: {membership}", f"Sector: {sector}", f"Market capitalisation: {mktCap}",
            f"Number of employees: {nb_ofemployees}", website]

###
# To display companies informations related to the volatility portfolios
@app.callback(
    [Output("company-image_2", "src"),
     Output("company_membership_2", "children"),
     Output("company-sector_2", "children"),
     Output("company-mktCap_2", "children"),
     Output("company-employees_2", "children"),
     Output("company-website_2", "href")],
    [Input("dropdown_2", "value")]
)
def company_info_vol_tab (value):
    if value is None:
        return [None,None,None,None,None,None]
    else:
        if value in company_names_high_vol:
            membership= "Portfolio with high volatility"
        elif value in company_names_low_vol:
            membership= "Portfolio with low volatility" 
        image = SP100_companies.image[(SP100_companies.companyName)== value].values[0]
        sector = SP100_companies.sector[(SP100_companies.companyName)== value].values[0]
        mktCap = SP100_companies.mktCap[(SP100_companies.companyName)== value].values[0]
        nb_ofemployees = SP100_companies.fullTimeEmployees[(SP100_companies.companyName)== value].values[0]
        website = SP100_companies.website[(SP100_companies.companyName)== value].values[0]
        return [image, f"Company membership: {membership}", f"Sector: {sector}", f"Market capitalisation: {mktCap}",
            f"Number of employees: {nb_ofemployees}", website] 
    
###
# This fuction is implemented to plot the line plot of the portfolios and then it is called inside
#the callback update_graph function
def line_plot(value, show_sp100):
    # Getting the portfolio data
    portfolio_df = globals()[value]
    
    # Creating figure for portfolio data
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=portfolio_df.index, y=portfolio_df["daily_discrete_PF_return"], name=value))
    
    # Adding SP100 index data if show_sp100 is True
    if "show_sp100" in show_sp100:
        fig.add_trace(go.Scatter(x=SP100_index_data.index, y=SP100_index_data["daily_discrete_return"], name="SP100 index"))
    
    fig.update_layout(
        title="Line plot of the daily discrete return",   
        xaxis_title="Date",
        yaxis_title="Daily Discrete Return",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    
    return fig

###
# Define callback to update graph
@app.callback(
    Output("graph_1", "figure"),
    [Input("dropdown_3", "value"),
     Input("checkbox_sp100", "value")],
)
def update_graph(value, show_sp100):
    fig = line_plot(value, show_sp100)
    return fig


###
# This function displays the distributions of the portfolios
@app.callback(
    Output("graph_2", "figure"),
    [Input("dropdown_4", "value")]
)
def histogramm(value):
    
    portfolio_df = globals()[value]
    fig = px.histogram(portfolio_df, x="daily_discrete_PF_return", color_discrete_sequence=['#FFA500'])
    fig.update_layout(
        title={
            'text': "Daily Discrete Return Histogram",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Daily Discrete Return",
        yaxis_title="Count"
    )
    return fig
          
    
# To update the Visit Website button visibility in the correlation tab   
@app.callback(
    Output("company-website_1", "style"),
    [Input("dropdown_1", "value")]
)
def corrTab_button_visibility(selected_company):
    if selected_company is None:
        return {"display": "none"}
    else:
        return {"display": "inline-block"}
  
    
# To update the Visit Website button visibility in the volatility tab   
@app.callback(
    Output("company-website_2", "style"),
    [Input("dropdown_2", "value")]
)
def VolTab_button_visibility(selected_company):
    if selected_company is None:
        return {"display": "none"}
    else:
        return {"display": "inline-block"}

    
    

app.run_server(debug=True,port=12345)

Dash app running on http://127.0.0.1:12345/
